In [1]:
import numpy as np
import numpy
import pandas as pd
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from levenberg_marquardt import LevenbergMarquardtReg

In [2]:
x_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtrain_0820.csv')
y_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytrain_competing_risk_0727.csv')
x_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtest_0820.csv')
y_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytest_competing_risk_0816_10y.csv')

In [3]:
x_train1_grs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_grs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]

In [4]:
x_train1_nogrs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_nogrs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]

In [5]:
y_train1_array_h = y_train1[["mof","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_h]
y_train1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_h = y_test1[["mof","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_h]
y_test1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_d = y_train1[["Death_10y","DeathDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_d]
y_train1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_d = y_test1[["Death_10y","DeathDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_d]
y_test1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_cr = y_train1[["mof_cr","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_cr]
y_train1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_cr = y_test1[["mof_cr","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_cr]
y_test1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [6]:
# Model 4 (FRAX CRFs + GRS)
estimator_h = ComponentwiseGradientBoostingSurvivalAnalysis(
    n_estimators = 153,
    learning_rate = 0.8503841871781403,
    subsample = 0.20011950297137547,
    random_state=0)

gcv_h = estimator_h.fit(x_train1_grs, y_train1_array_rsf_h)
gcv_d = ComponentwiseGradientBoostingSurvivalAnalysis().fit(x_train1_grs,y_train1_array_rsf_d)

score_train = gcv_h.predict(x_train1_grs)+gcv_d.predict(x_train1_grs)
score_train_h = pd.DataFrame(score_train, columns = ["TrainScore"])
train_cal=pd.concat([y_train1[["mof","WHOFRAC"]],score_train_h],axis="columns")
X, y = train_cal[["WHOFRAC"]].values, train_cal[["TrainScore"]].values
theta_actual = np.array([15., 0.1, 0.4]) 
sigma_actual = 0.6
    
def f(X, theta):
    return theta[0] * np.tanh(theta[1] + theta[2] * X[:, 0])
lr = LevenbergMarquardtReg(model_fn = f)    

lr.fit(X, y, theta_init = np.ones(3)) 
    
expected_values = lr.__get_optimization_status__(theta_actual)
print("\n*** RESULTS")
print("Estimated theta: {}".format(lr.theta))

Initial WSS: 87.66231646894332
Check after 10 iterations: % displacement = 1.099607686223397, norm_theta = 1.1604523432943188
Check after 20 iterations: % displacement = 0.0008970654763257878, norm_theta = 1.1442138308275869
Check after 30 iterations: % displacement = 0.0, norm_theta = 1.1442138308275869

*** RESULTS
Estimated theta: [ 3.83201501e-01  1.07813808e+00 -4.15246323e-04]


In [7]:
score_test = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs)

import math
sigmoid = lambda x: 1 / (0.383201501 + math.exp(1.07813808*x))-0.000415246323
sigmoid_v = np.vectorize(sigmoid) 

score_test_h = pd.DataFrame(sigmoid_v(score_test))
score_test_h.columns = ['SXGB']
score_test_h.to_csv('/users/PAS2433/dai417osc/WHI_sp23/data/SXGB_prob_mof.csv')